<a href="https://www.kaggle.com/jenniferturley/reading-a-large-file-assignment?scriptVersionId=88760543" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#import basic libraries 
import numpy as np
import pandas as pd
import time

#the dataset is a single file in csv format with delimiter ';'
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/bitcoin-tweets-20160101-to-20190329/tweets.csv


In [2]:
#install dask for faster loading (and for comparison with pandas.read_csv)
!pip -q install "dask[complete]" 

In [3]:
import dask.dataframe as dd
start_time = time.time()
#eliminating some columns to reduce memory usage for storage to to 4 GB
df = dd.read_csv('/kaggle/input/bitcoin-tweets-20160101-to-20190329/tweets.csv', low_memory = False, delimiter=';', skiprows=0, lineterminator='\n', usecols = ['user', 'fullname','url','timestamp','replies','likes','retweets'])
end_time = time.time()
print('Execution time, reading into dask df', end_time - start_time, 'seconds')

Execution time, reading into dask df 0.04599428176879883 seconds


In [4]:
# vaex overloads the available memory, so will not be attempted
#!pip install modin # modin is not found so modin will not be attempted

In [5]:
start_time = time.time()
df = pd.read_csv('/kaggle/input/bitcoin-tweets-20160101-to-20190329/tweets.csv', low_memory = False, delimiter=';', skiprows=0, lineterminator='\n', usecols = ['user', 'fullname','url','timestamp','replies','likes','retweets'])
end_time = time.time()
df.info(memory_usage = 'deep')
print('Execution time, reading into pandas df', end_time - start_time, 'seconds')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16889765 entries, 0 to 16889764
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   user       object
 1   fullname   object
 2   url        object
 3   timestamp  object
 4   replies    int64 
 5   likes      int64 
 6   retweets   int64 
dtypes: int64(3), object(4)
memory usage: 4.7 GB
Execution time, reading into pandas df 122.08043718338013 seconds


I tried to read the file with Vaex and Modin but Vaex exceeded my memory allotment and Modin was not compatible with this version of Kaggle. Vaex appears to need a lot of memory to read a file. Perhaps with a greater memory allotment, Vaex would perform well in terms of speed. However, the demands on memory of this method make it undesirable/impossible for this project on Kaggle, applied to this input data file. 

Dask and Pandas were successful for reading this csv and did so without exceeding the permitted memory allotment by Kaggle. However, the difference in terms of time taken is striking. Dask reads this csv file in under a second, while pandas read_csv requires about two minutes.

In [6]:
df.head()

,user,fullname,url,timestamp,replies,likes,retweets
0,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00,0,0,0
1,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0,0,0
2,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0,2,1
3,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0,0,0
4,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0,0,0


In [7]:
clist = df.columns.values.tolist()


In [8]:
%%writefile testutility.py
import logging
import subprocess
import yaml
import datetime 
import gc
import re

################
# File Reading #
################
filepath = '/kaggle/input/bitcoin-tweets-20160101-to-20190329/tweets.csv'
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


Writing testutility.py


In [9]:
filepath = '/kaggle/input/bitcoin-tweets-20160101-to-20190329/tweets.csv'
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

In [10]:
%%writefile file.yaml
file_type: csv
dataset_name: tweets
file_name: kaggle/input/bitcoin-tweets-20160101-to-20190329/tweets
table_name: Bitcoin_Tweets
inbound_delimiter: ";"
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - user
    - fullname
    - url
    - timestamp
    - replies
    - likes
    - retweets

Writing file.yaml


In [11]:

config_data = { 'file_type': 'csv',
                'dataset_name': 'tweets',
                'file_name': 'kaggle/input/bitcoin-tweets-20160101-to-20190329/tweets',
                'table_name': 'Bitcoin_Tweets',
                'inbound_delimiter': ';' ,
                'outbound_delimiter': '|' ,
                'skip_leading_rows': 1,
                'columns': ['user','fullname','url','timestamp','replies','likes','retweets']}

In [12]:
config_data

{'file_type': 'csv',
 'dataset_name': 'tweets',
 'file_name': 'kaggle/input/bitcoin-tweets-20160101-to-20190329/tweets',
 'table_name': 'Bitcoin_Tweets',
 'inbound_delimiter': ';',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['user',
  'fullname',
  'url',
  'timestamp',
  'replies',
  'likes',
  'retweets']}

In [13]:
df.head()

,user,fullname,url,timestamp,replies,likes,retweets
0,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00,0,0,0
1,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0,0,0
2,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0,2,1
3,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0,0,0
4,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0,0,0


In [14]:

file_type = config_data['file_type']
source_file = "/" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file, delimiter=config_data['inbound_delimiter'],usecols = ['user', 'fullname','url','timestamp','replies','likes','retweets'])
df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1,2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,user,fullname,url,timestamp,replies,likes,retweets
0,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00,0.0,0.0,0.0
1,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0.0,0.0,0.0
2,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0.0,2.0,1.0
3,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0.0,0.0,0.0
4,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0.0,0.0,0.0


In [15]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])


columns of files are: Index(['user', 'fullname', 'url', 'timestamp', 'replies', 'likes', 'retweets'], dtype='object')
columns of YAML are: ['user', 'fullname', 'url', 'timestamp', 'replies', 'likes', 'retweets']


In [16]:
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20165013 entries, 0 to 20165012
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   user       object 
 1   fullname   object 
 2   url        object 
 3   timestamp  object 
 4   replies    float64
 5   likes      float64
 6   retweets   float64
dtypes: float64(3), object(4)
memory usage: 5.4 GB


In [17]:
print('columns: ', len(df.columns))
print('rows: ', len(df))
print('number of elements: ', df.size)
print('4.7 GB memory used for storage as pd df')



columns:  7
rows:  20165013
number of elements:  141155091
4.7 GB memory used for storage as pd df


In [18]:
#convert to a compressed, gz file
import gzip
df.to_csv("BitcoinTweets.gz", sep = '|', compression="gzip")
print('The resulting .gz file is BitcoinTweets.gz(358.95 MB)')


The resulting .gz file is BitcoinTweets.gz(358.95 MB)
